In [ ]:
import gen3_validator
# loading gen3 bundled jsonschema into dictionary class
dd = gen3_validator.dict.DataDictionary('../examples/schema/json/schema_dev.json')
dd.parse_schema()

In [ ]:
# getting list of entities
dd.get_nodes()

In [ ]:
# Returning the jsonschema for a given entity
dd.return_schema('lipidomics_file.yaml')

In [ ]:
!pip install dictionaryutils

In [ ]:

import dictionaryutils
import json
import yaml
dd = dictionaryutils.DataDictionary('/Users/harrijh/projects/gen3schemadev/examples/schema/yaml')
dd.load_data(directory='/Users/harrijh/projects/gen3schemadev/examples/schema/yaml')
dd_resolved = dd.schema

for k, v in dd_resolved.items():
    with open(f'/Users/harrijh/projects/gen3schemadev/examples/schema/yaml/resolved/{k}.yaml', 'w') as f:
        yaml.safe_dump(v, f)


with open(f'/Users/harrijh/projects/gen3schemadev/examples/schema/json/gen3_bundled_schema_resolved.json', 'w') as f:
    json.dump(dd_resolved, f)

In [ ]:
dd = dictionaryutils.load_schemas_from_file('/Users/harrijh/projects/gen3schemadev/examples/schema/json/schema_dev.json')
dd

# To validate a gen3schemadev input yaml with the input schema


In [ ]:
# !pip install check-jsonschema

In [ ]:
!check-jsonschema --schemafile ../src/gen3schemadev/schema/input_schema.yml ../src/gen3schemadev/schema/input_example.yml --verbose

# To validate a single gen3 schema yaml with the metaschema
- note, the schema file needs to be resolved

In [ ]:
!check-jsonschema --schemafile ../src/gen3schemadev/schema/gen3_metaschema.yml ../examples/schema/yaml/resolved/lipidomics_file.yaml --verbose

if you give a non-resolved schema, it will not validate

In [ ]:
!check-jsonschema --schemafile ../src/gen3schemadev/schema/gen3_metaschema.yml ../examples/schema/yaml/lipidomics_file.yaml --verbose

# Compiling gen3schemadev input yaml to gen3 yamls
## Steps:
1. Validate input yaml against the input schema
2. Load the input yaml into python dictionary
3. The compiler should start with an empty data structure, with default values for a gen3 schema. This yaml is defined in the `src/gen3schemadev/schema/gen3_schema_template.yml`
4. 

In [1]:
from gen3schemadev.schema.gen3_template import *
from gen3schemadev.utils import *
from gen3schemadev.schema.input_schema import DataModel
import logging

# Set up basic logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

metaschema_path = "../src/gen3schemadev/schema/gen3_metaschema.yml"
converter_template = generate_gen3_template(metaschema_path)
metaschema = load_yaml(metaschema_path)

# loading input example
data = load_yaml('../tests/input_example.yml')
validated_model = DataModel.model_validate(data)


2025-10-01 16:25:47,899 [INFO] Successfully loaded YAML file: ../src/gen3schemadev/schema/gen3_metaschema.yml
2025-10-01 16:25:47,907 [INFO] Successfully loaded YAML file: ../src/gen3schemadev/schema/gen3_metaschema.yml
2025-10-01 16:25:47,910 [INFO] Successfully loaded YAML file: ../tests/input_example.yml


In [ ]:
converter_template

In [ ]:
metaschema

In [ ]:
validated_model.model_dump()

Now we need to read to:
1. validate the input yaml against the input metaschema
2. now we know the input yaml is validated, we can extract the data from the yaml into a data class which has the structure `input.entity.properties.links`
3. We then use the data class to populated the converter template for each entity
4. The populated templates are then written to the output directory


In [6]:
from dataclasses import dataclass
class Entity:
    name: str
    description: str
    category: str
    properties: list
    links: list

from typing import Any

def get_entity_data(entity: str, data: Any) -> Entity:
    try:
        for ent in data.entities:
            if ent.name == entity:
                return ent
        raise ValueError(f"Entity '{entity}' not found in data.entities")
    except AttributeError as e:
        raise AttributeError(f"Invalid data structure: {e}")
    except Exception as e:
        raise Exception(f"An error occurred while retrieving entity data: {e}")

def get_entity_links(entity: str, data: Any) -> list[dict]:
    links = data.links
    entity_links = []
    for link in links:
        if link.child == entity:
            entity_links.append(link.model_dump())
    return entity_links



from dataclasses import dataclass, asdict
from typing import Optional, List, Union

@dataclass
class LinkObj:
    name: str
    backref: str
    label: Optional[str]
    target_type: str
    multiplicity: str
    required: bool

    def to_dict(self):
        return asdict(self)

@dataclass
class LinkGroup:
    exclusive: bool
    required: bool
    subgroup: List[dict]

    def to_dict(self):
        return asdict(self)
    
def create_core_metadata_link(child_name: str) -> dict:
    link_obj = LinkObj(
        name=f"core_metadata_collections",
        backref=f"{child_name}s",
        label=None,
        target_type="core_metadata_collection",
        multiplicity="one_to_one",
        required=True 
    )
    return link_obj.to_dict()

def convert_entity_links(links: dict, entity_file: bool = False) -> dict:
    link_list = []
    for link in links:
        link_obj = LinkObj(
            name=f"{link['parent']}s",
            backref=f"{link['child']}s",
            label=None,
            target_type=link['parent'],
            multiplicity=link['multiplicity'],
            required=True  # TODO remove this hard code later, should pull from input yaml
        )
        link_list.append(link_obj.to_dict())

    if entity_file:
        core_link = create_core_metadata_link(links[0]['child'])
        link_list.append(core_link)

    if len(link_list) > 1:
        group = LinkGroup(
            exclusive=False,
            required=True,
            subgroup=link_list
        )
        output = group.to_dict()
    else:
        output = link_list
    return output

### Note to self, you will now need to find a way to add in the _definitions and _terms references.
### You also need to define a standard for the _definitions and _terms references files to read from as templates

def get_properties(entity_name: str, data: Any) -> list[dict]:
    output = []
    ent = get_entity_data(entity_name, data)
    props = ent.properties
    if props:
        for prop in props:
            pdict = {
                prop.name: {k: v for k, v in prop.model_dump().items() if k != "name"}
            }
            output.append(pdict)
    else:
        raise Exception(f'No properties found for entity {entity_name}')
    return output


def get_category(entity_name: str, data: Any) -> str:
    ent = get_entity_data(entity_name, data)
    category = ent.category
    # If it's an Enum, get its value; otherwise, return as is
    if hasattr(category, "value"):
        return category.value
    return category

def get_entity_value(entity_name: str, key: str, data: Any):
    """
    Returns the value of a single key within an entity object.

    Args:
        entity_name (str): The name of the entity to retrieve.
        key (str): The key whose value is to be returned.
        data (Any): The data structure containing entities.

    Returns:
        The value associated with the specified key in the entity object.
    """
    ent = get_entity_data(entity_name, data)
    return ent.model_dump()[key]


def populate_template(entity_name: str, input_data, template) -> dict:
    """
    Populate a Gen3 schema template dictionary with values from a Pydantic data model.

    This function takes an entity name, a Pydantic model instance containing entity data,
    and a Gen3 schema template dictionary. It fills a copy of the template with values
    from the input data, applying special logic for certain keys (e.g., 'name', 'category',
    'properties', 'links'). If a key from the input data is not found in the template,
    it is added with a value of None and a warning is logged.

    Args:
        entity_name (str): The name of the entity to populate in the template.
        input_data: A Pydantic model instance containing the entity's data.
        template (dict): A Gen3 schema template dictionary to be populated.

    Returns:
        dict: A new Gen3 schema template dictionary populated with values from the input data.

    Side Effects:
        Logs a warning if a key from the input data is not found in the template.
    """
    # ... function body ...
    ent = get_entity_data(entity_name, input_data)
    ent_dict = ent.model_dump()
    output_schema = template.copy()
    
    # Checking if entity is file category
    file_cat = False
    if get_entity_value(entity_name, 'category', input_data) == 'file':
        file_cat = True
    
    for key, value in ent_dict.items():
        if key == 'name':
            output_schema['id'] = value
        elif key == 'category':
            output_schema[key] = get_category(entity_name, input_data)
        elif key == 'properties':
            output_schema[key] = get_properties(entity_name, input_data)
        elif key == 'links':
            links = get_entity_links(entity_name, input_data)
            output_schema[key] = convert_entity_links(links, entity_file=file_cat)
        elif key in output_schema:
            output_schema[key] = value
        else:
            logger.warning(f"Key '{key}' not found in template")
    return output_schema

validated_model = DataModel.model_validate(data)
# links = get_entity_links('lipidomics_file', validated_model)
# convert_entity_links(links, entity_file=True)
# get_properties('sample', validated_model)
# get_category('sample', validated_model)

out_template = populate_template('lipidomics_file', validated_model, converter_template)
out_template
# write_yaml(out_template, 'output.yml')

Exception: No properties found for entity lipidomics_file

# Testing converter


In [20]:
from gen3schemadev.schema.gen3_template import *
from gen3schemadev.utils import *
from gen3schemadev.schema.input_schema import DataModel
from gen3schemadev.converter import *



# Loading template and metaschema
metaschema_path = "../src/gen3schemadev/schema/gen3_metaschema.yml"
converter_template = generate_gen3_template(metaschema_path)

# loading input example
data = load_yaml('../tests/input_example.yml')
validated_model = DataModel.model_validate(data)


out_template = populate_template('lipidomics_file', validated_model, converter_template)
write_yaml(out_template, 'output.yml')

2025-10-01 17:30:53,292 [INFO] Successfully loaded YAML file: ../src/gen3schemadev/schema/gen3_metaschema.yml


2025-10-01 17:30:53,305 [INFO] Successfully loaded YAML file: ../tests/input_example.yml
2025-10-01 17:30:53,314 [INFO] Successfully wrote YAML file: output.yml


In [2]:
from gen3schemadev.schema.gen3_template import *
from gen3schemadev.utils import *
from gen3schemadev.schema.input_schema import DataModel
from gen3schemadev.converter import *

data = load_yaml('../tests/input_example.yml')
validated_model = DataModel.model_validate(data)

from typing import Dict, Any, List

def strip_required_field(props_list: list[dict]) -> list[dict]:
    """
    Remove the 'required' field from all property dicts in the input list.
    Can use the output of get_properties() for this function

    Args:
        props_list (list): A list of property dictionaries, where each dictionary has a single key
            (the property name) and its value is a dictionary describing the property. For example:
                [
                    {
                        "project_id": {
                            "type": "string",
                            "description": "Synthetic_Dataset_1",
                            "required": True,
                            "enums": None
                        }
                    },
                    ...
                ]

    Returns:
        list: A new list with the same structure as props_list, but with the 'required'
            field removed from each property's dictionary (if present).

    Note:
        This function expects a list of property definitions as typically returned by
        get_properties() in the Gen3 schema conversion workflow.
        If you are working with a DataSourceProtocol object, you should first extract the
        properties list using the appropriate function.
    """
    new_list = []
    for prop in props_list:
        if isinstance(prop, dict):
            # Each prop is {property_name: property_dict}
            new_prop = {}
            for k, v in prop.items():
                if isinstance(v, dict):
                    v = {key: val for key, val in v.items() if key != 'required'}
                new_prop[k] = v
            new_list.append(new_prop)
        else:
            new_list.append(prop)
    return new_list

def get_required_prop_names(props_list: list[dict]) -> List[str]:
    """
    Given a list of property dicts (as from get_properties), return a list of property names
    where the property dict has 'required': True.

    Args:
        props_list (list): List of property dictionaries.

    Returns:
        List[str]: List of property names with required True.
    """
    required_names = []
    for prop in props_list:
        if isinstance(prop, dict):
            for k, v in prop.items():
                if isinstance(v, dict) and v.get("required") is True:
                    required_names.append(k)
    return required_names

project_props = get_properties('sample', validated_model)
stripped_props = strip_required_field(project_props)
required_names = get_required_prop_names(project_props)
stripped_props, required_names

2025-10-03 13:19:17,501 [INFO] Successfully loaded YAML file: ../tests/input_example.yml


([{'sample_id': {'type': 'string',
    'description': 'Sample ID (string)',
    'enums': None}},
  {'sample_count': {'type': 'integer',
    'description': 'Number of aliquots (integer)',
    'enums': None}},
  {'sample_volume': {'type': 'number',
    'description': 'Volume in microliters (number/float)',
    'enums': None}},
  {'is_viable': {'type': 'boolean',
    'description': 'Is the sample viable? (boolean)',
    'enums': None}},
  {'collection_date': {'type': 'datetime',
    'description': 'Date and time of collection (datetime)',
    'enums': None}},
  {'sample_tube_type': {'type': 'enum',
    'description': 'Sample tube type (enum)',
    'enums': [{'name': 'EDTA'}, {'name': 'Heparin'}, {'name': 'Citrate'}]}},
  {'notes': {'type': 'string',
    'description': 'Free text notes (string)',
    'enums': None}}],
 ['sample_id', 'collection_date'])

In [2]:
get_properties('lipidomics_file', validated_model)
construct_props('lipidomics_file', validated_model)


2025-09-30 16:35:46,694 [WARNING] No properties found for entity lipidomics_file
2025-09-30 16:35:46,695 [WARNING] No properties found for entity lipidomics_file


{'samples': {'$ref': '_definitions.yaml#/one_to_many'},
 'assays': {'$ref': '_definitions.yaml#/one_to_many'}}

In [2]:
get_entity_data('lipidomics_file', validated_model)

Entity(name='lipidomics_file', description='Info about lipidomics file', category=<CategoryEnum.DATA_FILE: 'data_file'>, properties=[])

In [2]:
links = get_entity_links('lipidomics_file', validated_model)
convert_entity_links(links, entity_file=True)
get_entity_value('lipidomics_file', 'category', validated_model) == 'data_file'

True

***
## Trying to create template from metaschema


In [ ]:
from gen3schemadev.schema.gen3_template import *

out_template = generate_gen3_template('../src/gen3schemadev/schema/gen3_metaschema.yml')

write_yaml(out_template, 'output.yml')

In [4]:
from gen3schemadev.schema.gen3_template import *
generate_def_template()
# generate_setting_template()
# generate_terms_template()
# generate_core_metadata_template()

{'id': '_definitions',
 'UUID': {'term': {'$ref': '_terms.yaml#/UUID'},
  'type': 'string',
  'pattern': '^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$'},
 'parent_uuids': {'type': 'array',
  'minItems': 1,
  'items': {'$ref': '#/UUID'},
  'uniqueItems': True},
 'foreign_key_project': {'type': 'object',
  'additionalProperties': True,
  'properties': {'id': {'$ref': '#/UUID'}, 'code': {'type': 'string'}}},
 'to_one_project': {'anyOf': [{'type': 'array',
    'items': {'$ref': '#/foreign_key_project', 'minItems': 1, 'maxItems': 1}},
   {'$ref': '#/foreign_key_project'}]},
 'to_many_project': {'anyOf': [{'type': 'array',
    'items': {'$ref': '#/foreign_key_project', 'minItems': 1}},
   {'$ref': '#/foreign_key_project'}]},
 'foreign_key': {'type': 'object',
  'additionalProperties': True,
  'properties': {'id': {'$ref': '#/UUID'},
   'submitter_id': {'type': 'string'}}},
 'to_one': {'anyOf': [{'type': 'array',
    'items': {'$ref': '#/foreign_key', 'minItem

# Testing yaml to bundle

In [2]:
from gen3schemadev.utils import bundle_yamls

bundle_yamls('../examples/schema/yaml')['subject']

2025-10-03 11:23:37,745 [INFO] Successfully loaded YAML file: ../examples/schema/yaml/demographic.yaml
2025-10-03 11:23:37,753 [INFO] Successfully loaded YAML file: ../examples/schema/yaml/project.yaml
2025-10-03 11:23:37,756 [INFO] Successfully loaded YAML file: ../examples/schema/yaml/acknowledgement.yaml
2025-10-03 11:23:37,759 [INFO] Successfully loaded YAML file: ../examples/schema/yaml/medical_history.yaml
2025-10-03 11:23:37,770 [INFO] Successfully loaded YAML file: ../examples/schema/yaml/_definitions.yaml
2025-10-03 11:23:37,772 [INFO] Successfully loaded YAML file: ../examples/schema/yaml/_settings.yaml
2025-10-03 11:23:37,773 [INFO] Successfully loaded YAML file: ../examples/schema/yaml/program.yaml
2025-10-03 11:23:37,777 [INFO] Successfully loaded YAML file: ../examples/schema/yaml/sample.yaml
2025-10-03 11:23:37,853 [INFO] Successfully loaded YAML file: ../examples/schema/yaml/_terms.yaml
2025-10-03 11:23:37,856 [INFO] Successfully loaded YAML file: ../examples/schema/yam

{'$schema': 'http://json-schema.org/draft-04/schema#',
 'id': 'subject',
 'title': 'Subject',
 'type': 'object',
 'namespace': 'http://commons.heartdata.baker.edu.au/',
 'category': 'clinical',
 'program': '*',
 'project': '*',
 'description': 'An individual participant in the study with baseline measurements.',
 'additionalProperties': False,
 'submittable': True,
 'validators': None,
 'systemProperties': ['id',
  'project_id',
  'state',
  'created_datetime',
  'updated_datetime'],
 'links': [{'name': 'projects',
   'backref': 'subjects',
   'label': 'part_of',
   'target_type': 'project',
   'multiplicity': 'many_to_one',
   'required': True}],
 'required': ['type', 'submitter_id', 'projects', 'baseline_year'],
 'uniqueKeys': [['id'], ['project_id', 'submitter_id']],
 'properties': {'$ref': '_definitions.yaml#/ubiquitous_properties',
  'projects': {'$ref': '_definitions.yaml#/to_one_project'},
  'consent_codes': {'description': 'Data Use Restrictions that are used to indicate  permi

In [3]:
import os
path = "../examples/schema/yaml/acknowledgement.yaml"

os.path.dirname(path)

'../examples/schema/yaml'

# Working on validation


In [ ]:
from gen3schemadev.utils import *
from gen3schemadev.schema.gen3_template import get_metaschema
import subprocess
import tempfile

def bundled_schema_to_dict_list(file: str, return_aux: bool = False):
    """
    Reads a bundled Gen3 JSON schema file and returns a list of schema dictionaries.

    Args:
        file (str): Path to the bundled JSON file containing multiple schemas.
        return_aux (bool): If True, return only the auxiliary schemas (definitions/settings/terms).
                           If False, return only the main entity schemas.

    Returns:
        list: A list of schema dictionaries extracted from the bundled file.
    """
    bundled = read_json(file)
    schema_list = []
    aux_list = []
    aux_schema_names = ['_definitions.yaml', '_settings.yaml', '_terms.yaml']
    for k, v in bundled.items():
        if k in aux_schema_names:
            aux_list.append(v)
        else:
            schema_list.append(v)

    if return_aux:
        return aux_list
    else:
        return schema_list


import jsonschema

def validate_schema_with_metaschema(schema, metaschema=None, verbose=False):
    """
    Validate a JSON Schema against a metaschema using the check-jsonschema CLI tool.

    This function writes the provided schema and metaschema to temporary files and
    invokes the external `check-jsonschema` command-line tool to perform validation.

    Args:
        schema (dict): The JSON Schema to validate.
        metaschema (dict, optional): The metaschema to validate against.
            If None, you must provide a metaschema explicitly.

    Raises:
        subprocess.CalledProcessError: If the check-jsonschema command fails.

    Returns:
        None. Raises an error if validation fails, otherwise completes silently.

    Note:
        This function does not return a value. It will raise an error if validation fails.
        It is intended for use in environments where the check-jsonschema CLI is available.
    """
    logger.info(f"Validating '{schema.get('id', '')}' with metaschema")
    # Create temp files for schema and metaschema
    with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as schema_file, \
         tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as metaschema_file:
        json.dump(schema, schema_file)
        schema_file.flush()
        json.dump(metaschema, metaschema_file)
        metaschema_file.flush()
        schema_path = schema_file.name
        metaschema_path = metaschema_file.name
    
    if verbose:
        cmd = [
            "check-jsonschema", "--verbose",
            "--schemafile", metaschema_path,
            schema_path
        ]
    else:
        cmd = [
            "check-jsonschema",
            "--schemafile", metaschema_path,
            schema_path
        ]
    # Capture output and wait for the subprocess to complete before returning
    completed_process = subprocess.run(cmd, capture_output=True, text=True)
    if completed_process.returncode != 0:
        logger.error(f"check-jsonschema failed with exit code {completed_process.returncode}")
        # Log stdout/stderr if available
        if completed_process.stdout:
            logger.error(f"STDOUT: {completed_process.stdout}")
        if completed_process.stderr:
            logger.error(f"STDERR: {completed_process.stderr}")
        # Do not raise an error, just return
        return



schema_list = bundled_schema_to_dict_list('../output/test_schema_bundle.json')
metaschema = get_metaschema()

for schema in schema_list:
    print(schema.get('id', ''))
    validate_schema_with_metaschema(schema, metaschema=metaschema, verbose=True)


2025-10-07 10:43:04,324 [INFO] Successfully loaded JSON file: ../output/test_schema_bundle.json
2025-10-07 10:43:04,331 [INFO] Validating 'project' with metaschema
2025-10-07 10:43:04,506 [INFO] Validating '_definitions' with metaschema


project
_definitions


2025-10-07 10:43:04,678 [ERROR] check-jsonschema failed with exit code 1
2025-10-07 10:43:04,678 [ERROR] STDOUT: Schema validation errors were encountered.
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpr6890yh7.json::$: 'title' is a required property
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpr6890yh7.json::$: 'type' is a required property
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpr6890yh7.json::$: 'category' is a required property
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpr6890yh7.json::$: 'program' is a required property
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpr6890yh7.json::$: 'project' is a required property
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpr6890yh7.json::$: 'description' is a required property
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpr6890yh7.json::$: 'submittable' is a required property
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpr6890yh7.json::$: 'validators' is a required proper


sample


2025-10-07 10:43:05,012 [ERROR] check-jsonschema failed with exit code 1
2025-10-07 10:43:05,012 [ERROR] STDOUT: Schema validation errors were encountered.
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpjzaet6m8.json::$.properties.sample_tube_type.type: 'enum' is not valid under any of the given schemas
  Underlying errors caused this.

  Best Match:
    $.properties.sample_tube_type.type: 'enum' is not one of ['array', 'boolean', 'integer', 'null', 'number', 'object', 'string']

  1 other errors were produced. Use '--verbose' to see all errors.
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpjzaet6m8.json::$.properties.collection_date.type: 'datetime' is not valid under any of the given schemas
  Underlying errors caused this.

  Best Match:
    $.properties.collection_date.type: 'datetime' is not one of ['array', 'boolean', 'integer', 'null', 'number', 'object', 'string']

  1 other errors were produced. Use '--verbose' to see all errors.
  /var/folders/h1/smw4rryj4zs361

_terms
lipidomics_file


2025-10-07 10:43:05,368 [ERROR] check-jsonschema failed with exit code 1
2025-10-07 10:43:05,368 [ERROR] STDOUT: Schema validation errors were encountered.
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmp4k3lggk8.json::$.links: {'exclusive': False, 'required': True, 'subgroup': [{'name': 'samples', 'backref': 'lipidomics_files', 'label': None, 'target_type': 'sample', 'multiplicity': 'one_to_many', 'required': True}, {'name': 'assays', 'backref': 'lipidomics_files', 'label': None, 'target_type': 'assay', 'multiplicity': 'one_to_many', 'required': True}, {'name': 'core_metadata_collections', 'backref': 'lipidomics_files', 'label': None, 'target_type': 'core_metadata_collection', 'multiplicity': 'one_to_one', 'required': True}]} is not of type 'array'

2025-10-07 10:43:05,368 [INFO] Validating 'assay' with metaschema
2025-10-07 10:43:05,543 [ERROR] check-jsonschema failed with exit code 1
2025-10-07 10:43:05,543 [ERROR] STDOUT: Schema validation errors were encountered.
  /var/fold

assay
core_metadata_collection


2025-10-07 10:43:05,734 [ERROR] check-jsonschema failed with exit code 1
2025-10-07 10:43:05,735 [ERROR] STDOUT: Schema validation errors were encountered.
  /var/folders/h1/smw4rryj4zs361v4bw9qqc0c0000gn/T/tmpa4illzze.json::$.properties['$ref']: '_definitions.yaml#/ubiquitous_properties' is not of type 'object'



In [3]:
schema_list[3]

{'$schema': 'http://json-schema.org/draft-04/schema#',
 'version': None,
 'id': 'sample',
 'title': 'sample',
 'type': 'object',
 'namespace': None,
 'category': 'clinical',
 'program': '*',
 'project': '*',
 'description': 'Info about sample',
 'submittable': True,
 'validators': None,
 'systemProperties': ['id',
  'project_id',
  'state',
  'created_datetime',
  'updated_datetime'],
 'uniqueKeys': [['id'], ['project_id', 'submitter_id']],
 'required': ['sample_id', 'collection_date'],
 'links': [{'name': 'projects',
   'backref': 'samples',
   'label': None,
   'target_type': 'project',
   'multiplicity': 'one_to_many',
   'required': True}],
 'properties': {'sample_id': {'type': 'string',
   'description': 'Sample ID (string)',
   'enums': None},
  'sample_count': {'type': 'integer',
   'description': 'Number of aliquots (integer)',
   'enums': None},
  'sample_volume': {'type': 'number',
   'description': 'Volume in microliters (number/float)',
   'enums': None},
  'is_viable': {'t

In [6]:
schema_list

[]